# Шахматное программирование

In [1]:
import os

## Вспомогательные функции

In [2]:
BASE_FOLDER = '../../algorithms_course_materials/homework_4/2019-08-12_Chess-Tasks'
ASSIGNMENTS = sorted(os.listdir(BASE_FOLDER))
# FEN_EXAMPLE = 'rnbqkbnr/pppp1ppp/8/1N6/4pP2/8/PPPPP1PP/R1BQKBNR b KQkq f3 0 3'

In [3]:
def get_folder(n):
    return ASSIGNMENTS[n - 1]


def get_assignment(n):
    with open(os.path.join(BASE_FOLDER, get_folder(n), 'problem.txt')) as f:
        print(f.read())

       
def get_inputs(n):
    dirname = ASSIGNMENTS[n - 1]
    result = []
    for filename in os.listdir(os.path.join(BASE_FOLDER, dirname)):
        if 'test' in filename:
            base, ext = os.path.splitext(filename)
            if ext == '.in':
                with open(os.path.join(BASE_FOLDER, dirname, filename)) as f:
                    result.append(f.read().strip())
    return result


def get_fen_from_file(hw_num, num, out=True):
    if out:
        filepath = os.path.join(BASE_FOLDER, get_folder(hw_num), f'test.{num}.out')
    else:
        filepath = os.path.join(BASE_FOLDER, get_folder(hw_num), f'test.{num}.in')
    with open(filepath) as f:
        return f.read().strip()


def get_ascii_from_file(hw_num, num):
    filepath = os.path.join(BASE_FOLDER, get_folder(hw_num), f'test.{num}.out')
    with open(filepath) as f:
        return f.read().rstrip('\n')

## Основной код

In [4]:
class Chess:
    
    def __init__(self):
        self.placement = {}
        self.is_white = True
        self.castling = '-'
        self.en_passant = '-'
        self.halfmove_clock = 0
        self.fullmove_number = 0
        self.move = ''
        
    def load_fen_from_file(self, hw_num, num):
        filepath = os.path.join(BASE_FOLDER, get_folder(hw_num), f'test.{num}.in')
        with open(filepath) as f:
            fen = f.read().split('\n')
        self.load_fen(fen[0])
        if len(fen) > 1:
            if fen[1]:
                self.move = fen[1]
                self.count_fullmoves()
                self.count_halfmoves()
        
    def count_fullmoves(self):
        if self.is_white:
            self.is_white = False
        else:
            self.is_white = True
            self.fullmove_number += 1
    
    def count_halfmoves(self):
        if self.placement[self.move[:2]].lower() == 'p':
            self.halfmove_clock = 0
        elif self.placement[self.move[2:]] != '.':
            self.halfmove_clock = 0
        else:
            self.halfmove_clock += 1
        
    def load_fen(self, fen):
        fen_list = fen.split()
        self.is_white = self.parse_color(fen_list[1])
        self.castling = self.parse_castling(fen_list[2])
        self.en_passant = fen_list[3]
        self.halfmove_clock = int(fen_list[4])
        self.fullmove_number = int(fen_list[5])
        self.parse_placement(fen_list[0])
        
    def parse_placement(self, placement_string):
        lines = placement_string.split('/')
        lines = [self.line_transform(line) for line in lines]
        for digit in range(1, 9):
            for i, letter in enumerate('abcdefgh'):
                key = letter + str(digit)
                value = lines[8 - digit][i]
                self.placement[key] = value
                    
    @staticmethod
    def line_transform(line):
        result = ''
        for cell in line:
            try:
                result += '.' * int(cell)
            except ValueError:
                result += cell
        if len(result) != 8:
            result += '.' * (8 - len(result))
        return result

    def parse_color(self, color):
        if color.lower() != 'w':
            return False
        return True
    
    def parse_castling(self, castling):
        return ''.join(sorted(castling))
        
    def get_fen_placement(self):
        fen = []
        for digit in range(8, 0, -1):
            line = ''
            dot_count = 0
            for i, letter in enumerate('abcdefgh'):
                key = letter + str(digit)
                value = self.placement[key]
                if value == '.':
                    dot_count += 1
                if value != '.' and not dot_count:
                    line += value
                if value != '.' and dot_count:
                    line += str(dot_count)
                    line += value
                    dot_count = 0
            if dot_count:
                line += str(dot_count)
            fen.append(line)
        return '/'.join(fen)
    
    def get_fen(self):
        placement = self.get_fen_placement()
        if self.is_white:
            placement += ' w'
        else:
            placement += ' b'
        placement += ' ' + self.castling + ' ' + self.en_passant
        placement += ' ' + str(self.halfmove_clock) + ' ' + str(self.fullmove_number)
        return placement
    
    def get_base_board(self):
        board = []
        for digit in range(8, 0, -1):
            line = ''
            for i, letter in enumerate('abcdefgh'):
                key = letter + str(digit)
                line += self.placement[key]
            board.append(' '.join(line))
        return board
                
    def show_ascii_board(self):
        board = []
        border_line = '  +' + '-' * 17 + '+'
        board.append(border_line)
        base_board = self.get_base_board()
        for i, line in enumerate(base_board):
            board.append(str(8 - i) + ' | ' + line + ' |')
        board.append(border_line)
        board.append('    a b c d e f g h  ')
        return '\n'.join(board)

## 1. FEN - ASCII

In [5]:
get_assignment(1)

1.FEN - ASCII

Дано расположение шахматных фигур на доске в FEN-нотации.

Вывести её в текстовом ASCII формате по образцу.
На диаграмме должно присутствовать:
<ul><li>рамка вокруг позиции, </li><li>буквы <span class='prog'>a-h</span> снизу,</li><li>цифры <span class='prog'>1-8</span> слева,</li> <li>точки на пустых полях,</li> <li>фигуры на своих местах</li></ul>

<strong>Начальные данные</strong>: строка символов - позиция в FEN нотации
<strong>Вывод результата</strong>: 11 строчек по 21 символу на каждом.



### Пример выполнения

In [6]:
c = Chess()
c.load_fen_from_file(1, 4)
print(c.get_fen())
print()
print(c.show_ascii_board())

r1n1k1b1/1r1b1q1n/1p1p1p1p/p1p1p1p1/1P1P1P1P/P1P1P1P1/1R1B1Q1N/R1N1K1B1 w - - 0 50

  +-----------------+
8 | r . n . k . b . |
7 | . r . b . q . n |
6 | . p . p . p . p |
5 | p . p . p . p . |
4 | . P . P . P . P |
3 | P . P . P . P . |
2 | . R . B . Q . N |
1 | R . N . K . B . |
  +-----------------+
    a b c d e f g h  


### Тесты

In [7]:
c = Chess()

flag = True
for i in range(10):
    c.load_fen_from_file(1, i)
    ascii_from_file = get_ascii_from_file(1, i)
    if c.show_ascii_board() != ascii_from_file:
        flag = False
        print(i)
        print(c.show_ascii_board())
        print(ascii_from_file)
        print()
if flag:
    print('\nВсе тесты пройдены')


Все тесты пройдены


## 2. FEN - BITS *

In [8]:
#get_assignment(2)

## 3. Сборка и разборка

In [9]:
get_assignment(3)

1.Сборка и разборка

Создать структуру для хранения позиции.
Написать функцию для парсинга FEN-позиции в эту структуру.
Написать функцию для формирования FEN-строки из этой структуры.

Дано: FEN-позиция, записанная с небольшими неточностями.
Надо: FEN-строка, созданная по данной позиции.



### Пример выполнения

In [10]:
c = Chess()

c.load_fen_from_file(3, 1)
fen_from_file = get_fen_from_file(3, 1, out=False)
fen_processed = c.get_fen()

print(fen_from_file)
print(fen_processed)

k1111111/2111111/311111/41111/5111/611/17/16K w - - 0 70
k7/8/8/8/8/8/8/7K w - - 0 70


### Тесты

In [11]:
c = Chess()

flag = True
for i in range(10):
    c.load_fen_from_file(3, i)
    fen_processed = c.get_fen()
    fen_from_file = get_fen_from_file(3, i)
    if fen_processed != fen_from_file:
        flag = False
        print(i)
        print(fen_processed)
        print(fen_from_file)
        print()
if flag:
    print('\nВсе тесты пройдены')


Все тесты пройдены


## 4. Счётчик ходов

In [12]:
get_assignment(4)

1.Счётчик ходов

Дана FEN-позиция и ход фигурой.
Нужно посчитать этот ход и передать право хода другой стороне.
ФИГУРЫ ПЕРЕМЕЩАТЬ НЕ НУЖНО

Дано: 
   FEN-позиция
   ход фигурой
Надо:
   FEN-позиция после хода


### Тесты

In [13]:
c = Chess()

flag = True
for i in range(5):
    c.load_fen_from_file(4, i)
    fen_processed = c.get_fen()
    fen_from_file = get_fen_from_file(4, i)
    if fen_processed != fen_from_file:
        flag = False
        print(i)
        print(fen_processed)
        print(fen_from_file)
        print()
if flag:
    print('\nВсе тесты пройдены')

2
k7/8/8/8/8/8/8/K7 b - - 91 70
k7/8/8/8/8/8/8/K7 b - - 90 70

3
k7/8/8/8/8/8/8/K7 w - - 91 71
k7/8/8/8/8/8/8/K7 w - - 90 71

4
r1bqkbnr/ppp1pppp/2n5/3p4/4P3/5N2/PPPP1PPP/RNBQKBR1 w Qkq - 0 4
r1bqkbnr/ppp1pppp/2n5/3p4/4P3/5N2/PPPP1PPP/RNBQKBR1 w Qkq - 3 4



Ошибки в тестах 2, 3, 4 из-за различия в подсчете количества полуходов из-за его имплементации в задании 5

## 5. Счётчик полуходов

In [14]:
get_assignment(5)

1.Счётчик полуходов

Дана FEN-позиция и ход фигурой.
Нужно посчитать этот ход, передать право хода другой стороне,
а также увеличить или сбросить счётчик полуходов для правила 50 ходов.
Счётчик сбрасывается, если было взятие или ход пешкой.
В остальных случаях счётчик увеличивается на 1 после каждого полухода.
ФИГУРЫ ПЕРЕМЕЩАТЬ НЕ НУЖНО

Дано: 
   FEN-позиция
   ход фигурой
Надо:
   FEN-позиция после хода


### Тесты

In [15]:
c = Chess()

flag = True
for i in range(10):
    c.load_fen_from_file(5, i)
    fen_processed = c.get_fen()
    fen_from_file = get_fen_from_file(5, i)
    if fen_processed != fen_from_file:
        flag = False
        print(i)
        print(fen_processed)
        print(fen_from_file)
        print()    
if flag:
    print('\nВсе тесты пройдены')


Все тесты пройдены


## 6. Перемещение фигуры

In [16]:
get_assignment(6)

1.Перемещение фигуры

Дана FEN-позиция и ход фигурой.
Это обычных ход или взятие.
Не рокировка, не взятие на проходе, не превращение ферзя, а обычный, самый простой ход со взятием или без.

Необходимо переставить фигуры и передать право хода.
Делать проверку на возможность хода не нужно.

Дано: 
   FEN-позиция
   ход фигурой
Надо:
   FEN-позиция после хода
